In [259]:
import networkx as nx
import numpy as np
import pandas as pd
import pymatgen
import pymatgen.io.xyz

In [260]:
benzene_automorphisms = (((0,), (1,),(2,),(3,),(4,),(5,)), 
    ((0,1,2,3,4,5),),
    ((5,4,3,2,1,0),),
    ((0,2,4), (1,3,5)),
    ((4,2,0), (5,3,1)),
    ((0,3), (1,4), (2,5)),
    ((0,), (3,), (1,5), (2,4)),
    ((2,), (5,), (0,4), ((1,3))),
    ((1,), (4,), (0,2), (3,5)),
    ((0,5), (1,4),(2,3)),
    ((0,1), (2,5), (3,4)),
    ((0,3), (1,2), (4,5)))

def apply_automorphisms(zs, automorphisms):
    for automorphism in automorphisms:
        mapping = np.zeros(6, dtype=int)
        for cycle in automorphism:
            if len(cycle) == 1:
                mapping[cycle[0]] = cycle[0]
            elif len(cycle) == 2:
                mapping[cycle[1]] = cycle[0]
                mapping[cycle[0]] = cycle[1]
            else:
                mapping[list(cycle[1:])] = cycle[:-1]
                mapping[cycle[0]] = cycle[-1]
        yield np.array(zs)[mapping]

In [201]:
def generate(based_on, add):
    modifiable = np.where(np.array(based_on) == 6)[0]
    found = []
    for pos in modifiable:
        zsnew = based_on.copy()
        zsnew[pos] = add
        if is_canonical(zsnew):
            found.append(zsnew)
    return found

In [216]:
def add_bnpair(zs):
    for i in generate(zs, 5):
        print (i)
        for j in generate(i, 7):
            yield j

In [203]:
start = np.array((6,6,6,6,6,6), dtype=int)
res = []
for k in add_bnpair(start):
    print (k)
    for kk in add_bnpair(k):
        print (k, "parent of ", kk)
        res.append(tuple(kk))
print (len(res), len(set(res)))


[7 6 6 6 6 5]
[7 6 6 6 6 5] parent of  [7 5 7 6 6 5]
[7 6 6 6 6 5] parent of  [7 5 6 7 6 5]
[7 6 6 6 6 5] parent of  [7 7 5 6 6 5]
[7 6 6 6 6 5] parent of  [7 6 5 7 6 5]
[7 6 6 6 6 5] parent of  [7 7 6 5 6 5]
[7 6 6 6 6 5] parent of  [7 6 7 5 6 5]
[7 6 6 6 6 5] parent of  [7 7 6 6 5 5]
[7 6 6 6 6 5] parent of  [7 6 7 6 5 5]
[7 6 6 6 6 5] parent of  [7 6 6 7 5 5]
[6 7 6 6 6 5]
[6 7 6 6 6 5] parent of  [7 7 6 5 6 5]
[6 7 6 6 6 5] parent of  [7 7 6 6 5 5]
[6 7 6 6 6 5] parent of  [6 7 7 6 5 5]
[6 6 7 6 6 5]
12 10


In [204]:
def make_canonical(zs):
    zs = np.array(zs)
    #if len(np.where(zs == 7)[0]) > len(np.where(zs == 5)[0]):
    #    return False
    permutations = np.array([_ for _ in apply_automorphisms(zs, benzene_automorphisms)])
    o = np.lexsort(permutations.T)
    canonical = permutations[o[0]]
    return canonical
def is_canonical(zs):
    return (zs == make_canonical(zs)).all()

In [214]:
targets = ((6,7,7,6,5,5), (7,7,5,5,6,6), (7,7,5,6,5,6), (7,6,7,5,5,6), (5,6,5,7,6,7), (6,6,7,5,5,7), (6,6,5,7,7,5), (7,6,5,7,5,6), (5, 6, 7, 5, 7, 6), (6,6,7,5,7,5), (6,7,5,6,7,5))
for target in targets:
    print (target, tuple(make_canonical(target)) in res)


(6, 7, 7, 6, 5, 5) True
(7, 7, 5, 5, 6, 6) True
(7, 7, 5, 6, 5, 6) True
(7, 6, 7, 5, 5, 6) True
(5, 6, 5, 7, 6, 7) True
(6, 6, 7, 5, 5, 7) True
(6, 6, 5, 7, 7, 5) True
(7, 6, 5, 7, 5, 6) True
(5, 6, 7, 5, 7, 6) False
(6, 6, 7, 5, 7, 5) True
(6, 7, 5, 6, 7, 5) True


In [173]:
np.where(np.array((7,6,6,6,6,6)) == 7)

(array([0]),)

In [215]:
make_canonical((5, 6, 7, 5, 7, 6))

array([6, 7, 5, 7, 6, 5])

In [219]:
for i in add_bnpair(np.array([6, 7, 6, 6, 6, 5])):
    pass

[6 7 6 5 6 5]
[6 7 6 6 5 5]


In [220]:
make_canonical((5,7,6,6,6,5))

array([7, 6, 6, 6, 5, 5])

In [250]:
def two_at_once(based_on):
    modifiable = sorted(np.where(np.array(based_on) == 6)[0])
    try:
        limit5 = max(np.where(np.array(based_on) == 5)[0])
    except:
        limit5 = 100
    try:
        limit7 = max(np.where(np.array(based_on) == 7)[0])
    except:
        limit7 = 100
    for i in modifiable:
        #if i >= limit5:
        #    continue
        for j in modifiable:
            #if j >= limit7:
            #    continue
            if i == j:
                continue
            zsnew = np.array(based_on).copy()
            zsnew[i] = 5
            zsnew[j] = 7
            
            if is_canonical(zsnew):
                yield zsnew
res = []
for k in two_at_once((6,6,6,6,6,6)):
    print ("level 1", k)
    for kk in two_at_once(k):
        print (kk, tuple(kk) in res)
        res.append(tuple(kk))
print (len(set(res)), len(res))

level 1 [7 6 6 6 6 5]
[7 5 7 6 6 5] False
[7 5 6 7 6 5] False
[7 7 5 6 6 5] False
[7 6 5 7 6 5] False
[7 7 6 5 6 5] False
[7 6 7 5 6 5] False
[7 7 6 6 5 5] False
[7 6 7 6 5 5] False
[7 6 6 7 5 5] False
level 1 [6 7 6 6 6 5]
[7 7 5 6 6 5] True
[6 7 5 7 6 5] False
[7 7 6 5 6 5] True
[7 7 6 6 5 5] True
[6 7 7 6 5 5] False
level 1 [6 6 7 6 6 5]
[7 5 7 6 6 5] True
[7 6 7 5 6 5] True
[7 6 7 6 5 5] True
[6 7 7 6 5 5] True
11 18


In [245]:
(np.array((1,2,3)) < np.array((1,3,4))).all()

False

In [317]:
def detect_automorphisms(filename):
    xyz = pymatgen.io.xyz.XYZ.from_file(filename)
    psa = pymatgen.symmetry.analyzer.PointGroupAnalyzer(xyz.molecule)
    
    m = xyz.molecule.get_centered_molecule()
    carbons = np.where(np.array(m.atomic_numbers, dtype=np.int) == 6)[0]
    
    operations = psa.get_symmetry_operations()
    mapping = np.zeros((len(carbons), len(operations)), dtype=np.int)
    for opidx, op in enumerate(operations):
        for bidx, base in enumerate(carbons):
            ds = np.linalg.norm(op.operate(m.cart_coords[base]) - m.cart_coords[carbons], axis=1)
            onto = np.argmin(ds)
            if ds[onto] > 1e-3:
                raise ValueError('Irregular geometry')
            mapping[bidx, opidx] = onto
    
    return mapping


    
    return xyz.molecule, psa
detect_automorphisms("../../test/benzene.xyz")

array([[5, 1, 0, 4, 0, 5, 2, 0, 1, 3, 5, 4, 0, 3, 1, 2, 2, 4, 3, 5, 3, 1,
        2, 4],
       [0, 0, 1, 5, 5, 0, 1, 1, 0, 4, 4, 5, 5, 2, 2, 1, 3, 3, 4, 4, 2, 2,
        3, 3],
       [1, 5, 2, 0, 4, 1, 0, 2, 5, 5, 3, 0, 4, 1, 3, 0, 4, 2, 5, 3, 1, 3,
        4, 2],
       [2, 4, 3, 1, 3, 2, 5, 3, 4, 0, 2, 1, 3, 0, 4, 5, 5, 1, 0, 2, 0, 4,
        5, 1],
       [3, 3, 4, 2, 2, 3, 4, 4, 3, 1, 1, 2, 2, 5, 5, 4, 0, 0, 1, 1, 5, 5,
        0, 0],
       [4, 2, 5, 3, 1, 4, 3, 5, 2, 2, 0, 3, 1, 4, 0, 3, 1, 5, 2, 0, 4, 0,
        1, 5]])

In [305]:
s = q.get_symmetry_operations()[3]

In [313]:
for base in carbons:
    ds = np.linalg.norm(s.operate(m.cart_coords[base]) - m.cart_coords[carbons], axis=1)
    onto = np.argmin(ds)
    if ds[onto] > 1e-3:
        raise ValueError('Irregular geometry')
    print (base, "->", onto)
    
    

0 -> 4
1 -> 5
2 -> 0
3 -> 1
4 -> 2
5 -> 3


In [308]:
m.cart_coords[0], m.cart_coords[2], m.cart_coords[4]

(array([-0.75007667,  1.18097487,  0.        ]),
 array([ 1.39779333,  0.05910487, -0.        ]),
 array([-0.64771667, -1.24007513,  0.        ]))

4